# CHAPTER 14: TensorBoard - Big Brother of TensorFlow

## Ringkasan

Chapter ini membahas **TensorBoard**, toolkit visualisasi powerful untuk memantau, mendebug, dan menganalisis kinerja model machine learning. TensorBoard memungkinkan visualisasi real-time metrik training (loss, accuracy), histogram aktivasi layer untuk mendeteksi masalah konvergensi, profiling performa untuk menemukan bottleneck komputasi, dan proyeksi embedding untuk menganalisis data dimensi tinggi seperti word vectors. Bab mencakup visualisasi image Fashion-MNIST, tracking metrik training untuk model dense dan CNN, logging metrik custom dengan API tf.summary, profiling performa dengan rekomendasi optimisasi (mixed precision training, perbaikan tf.data pipeline), dan visualisasi word vector dengan embedding GloVe pada dataset IMDB.

---

## Bagian 1: Visualisasi Data dengan TensorBoard

### Dataset Fashion-MNIST

Fashion-MNIST adalah pengganti modern untuk klasifikasi digit MNIST—task yang lebih menantang dengan format sama (gambar 28×28 skala abu-abu, 10 kelas). Kategori: Kaos/atas, Celana, Pullover, Gaun, Mantel, Sandal, Kemeja, Sneaker, Tas, Boot pergelangan kaki. Dataset diunduh via library `tensorflow_datasets`, mengembalikan split train/test dengan format dictionary `{image: (28,28,1), label: int}`.

### Setup Pipeline Data

Pipeline mengkonversi record dictionary ke format tuple untuk kompatibilitas training model. Langkah-langkah kunci:
1. **Shuffle dan Konversi**: Data training di-shuffle (buffer 20×batch_size) dan dikonversi dari dict ke tuple `(image, label)`
2. **Flatten Opsional**: Untuk fully connected networks, gambar 2D (28×28) di-flatten ke vektor 1D (784)
3. **Split Train/Validasi**: 10.000 pertama dari training set menjadi validasi, sisanya training

### Logging Gambar ke TensorBoard

**tf.summary.SummaryWriter**: Mekanisme inti untuk menulis data ke direktori log. Langkah-langkah:
1. **Buat writer**: `tf.summary.create_file_writer(log_dir)` dengan direktori yang ditimestamp
2. **Buka context**: `with writer.as_default():` untuk mengaktifkan writer
3. **Tulis gambar**: `tf.summary.image(name, data, step, max_outputs)` untuk log batch gambar

**Argumen**:
- `name`: Tag untuk pengelompokan gambar di dashboard
- `data`: Tensor dengan shape `[batch, height, width, channels]`
- `step`: Integer untuk tracking iterasi berbeda
- `max_outputs`: Maksimal gambar untuk ditampilkan (default 3)

**Dua pendekatan logging**:
1. **Gambar individual dengan label class**: Loop melalui 10 gambar, log masing-masing dengan label sebagai tag—gambar dikelompokkan berdasarkan class
2. **Logging batch**: Log 20 gambar sekaligus dalam satu batch—ditampilkan bersama

### Meluncurkan TensorBoard

**Di Jupyter Notebook**:
```python
%load_ext tensorboard
%tensorboard --logdir ./logs --port 6006
```

**Browser Standalone**: Navigate ke `http://localhost:6006` setelah command notebook dijalankan.

Fitur dashboard: slider brightness/contrast, pemilihan subdirectory (toggle visibility), inspeksi gambar, organisasi run dengan timestamp.

---

## Bagian 2: Tracking dan Monitoring Performa Model

### Definisi Model

**Dense Network**:
- 3 layer: 512 neuron (ReLU), 256 neuron (ReLU), 10 output (softmax)
- Input: Vektor 784-D yang di-flatten
- Loss: Sparse categorical cross-entropy
- Optimizer: Adam
- Metrics: Accuracy

**CNN Network**:
- Conv2D (32 filter, kernel 5×5, stride 2×2, ReLU)
- Conv2D (16 filter, kernel 3×3, stride 1×1, ReLU)
- Flatten
- Dense (10 output, softmax)
- Input: Gambar 28×28×1
- Setelan kompilasi sama

### TensorBoard Callback

**tf.keras.callbacks.TensorBoard()**: Callback built-in Keras untuk automatic logging. Argumen penting:

- **log_dir**: Direktori untuk menulis log (default 'logs'). Gunakan subdirectory dengan timestamp untuk unique run
- **histogram_freq**: Interval (dalam epoch) untuk log histogram aktivasi layer (0 = disabled)
- **write_graph**: Tulis arsitektur model graph (default True)
- **update_freq**: Frekuensi penulisan—'batch', 'epoch', atau integer (interval batch)
- **profile_batch**: Range batch untuk profiling (integer atau [start, end])
- **embeddings_freq**: Interval untuk visualisasi embedding (0 = disabled)

**Penggunaan**:
```python
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, profile_batch=0
)
model.fit(train_ds, validation_data=valid_ds, epochs=10, callbacks=[tb_callback])
```

### Organisasi Run

**Praktik Terbaik**: Struktur direktori berhirarki dengan timestamp untuk membedakan experiment. Contoh:

**Flat dengan timestamp**:
```
./logs/dense_20210527031421
./logs/dense_20210527090252
./logs/conv_20210527101209
```

**Nested berdasarkan hyperparameter**:
```
./logs/dense/lr=0.01/20210527031421
./logs/dense/lr=0.001/20210527090252
./logs/conv/filters=32/20210527101209
```

Timestamp memastikan folder unique dan memudahkan perbandingan historis.

### Kontrol Dashboard

**Smoothing Parameter**: Slider (0.0 hingga 0.99) untuk smooth metrik noisy, mengungkap tren mendasar. Nilai lebih tinggi = garis lebih smooth, original ditampilkan faded di background.

**Y-axis Scale**: Toggle linear ↔ log scale untuk visualisasi lebih baik ketika metrik spanning multiple order magnitude.

**Pemilihan Run**: Checkbox untuk show/hide run berbeda, enabling perbandingan terfokus.

**Ukuran Plot**: Toggle standard ↔ full-size untuk pemeriksaan detail.

**Auto-fit**: Kontrol zoom untuk fit garis dalam area plot.

### Histogram Aktivasi

**Purpose**: Visualisasi distribusi aktivasi layer sepanjang epoch training. Kritis untuk mendeteksi masalah konvergensi, dead neuron, exploding/vanishing gradient.

**Enable**: Set `histogram_freq=2` (log setiap 2 epoch) dalam TensorBoard callback.

**Interpretasi**:
- **Distribusi converging**: Weight settling dalam pola stabil (typically approximately normal)
- **Distribusi multi-modal**: Multiple peaks mengindikasikan diverse activation pattern
- **Exploding distribution**: Weight shifting ke nilai extreme (training instability)
- **Dead neuron**: Konsentrasi besar di zero (ReLU saturation)

**Visualisasi**: Histogram 3D berstacked dengan warna lebih terang = epoch lebih recent. X-axis = binned weight value, Y-axis = count. TensorBoard menggunakan exponential bin (fine-grained dekat zero, wider bin jauh dari zero) dan resampling untuk uniform visualization.

---

## Bagian 3: Custom Metric Logging dengan tf.summary

### Motivasi: Menganalisis Efek Batch Normalization

**Experiment**: Bandingkan statistik weight (mean, std) dari Dense layer dengan dan tanpa batch normalization sepanjang training step.

**Model**:
1. **Standard Dense**: 512 → 256 → 10 neuron (track layer 'log_layer')
2. **Dense dengan BatchNorm**: 512 → BatchNorm → 256 → BatchNorm → 10 (track 'log_layer_bn')

### Custom Training Loop

**Mengapa Custom Loop**: Metrik standard Keras tidak include statistik weight spesifik-layer. Perlu komputasi manual dan logging setiap batch.

**Langkah-langkah Implementasi**:
1. **Buat writer**: `tf.summary.create_file_writer(log_dir)`
2. **Training loop**: Iterasi epoch → batch
3. **Per-batch operation**:
   - `model.train_on_batch(x, y)`: Update weight
   - Extract weight: `model.get_layer(name).get_weights()[0]` (index 0 = weight, 1 = bias)
   - Hitung metrik: `np.mean(np.abs(weights))`, `np.std(np.abs(weights))`
   - Log scalar: `tf.summary.scalar('mean_weights', value, step=step)`
4. **Flush buffer**: `writer.flush()` untuk menulis dari memory ke disk

### tf.summary API

**Tipe umum**:
- `tf.summary.scalar()`: Nilai numerik tunggal (loss, accuracy, metrik custom)
- `tf.summary.histogram()`: Visualisasi distribusi
- `tf.summary.image()`: Data gambar
- `tf.summary.audio()`: File audio
- `tf.summary.text()`: Data teks raw

**Argumen**:
- `name`: Identifier metrik (ditampilkan di dashboard)
- `data`: Value untuk di-log
- `step`: Integer counter step (x-axis di dashboard)

### Interpretasi Hasil

Dengan batch normalization, mean dan std dari weight vary **significantly lebih banyak** dibanding model standard. Alasan: Normalisasi eksplisit antara layer memberikan weight lebih kebebasan bergerak selama optimisasi, karena batch norm re-scale aktivasi.

---

## Bagian 4: Performance Profiling

### Dataset Klasifikasi Bunga

Dataset flower 17-kategori (https://www.robots.ox.ac.uk/~vgg/data/flowers) dengan gambar diorganisir berdasarkan numbering filename: 80 gambar pertama = class 0, 80 berikutnya = class 1, dll. Data pipeline membaca file .jpg, extract label dari filename, resize ke 64×64, membuat train/valid split.

### Arsitektur CNN Model

**Layer**:
- Conv2D (64 filter, 5×5, stride 1, ReLU) + BatchNorm + MaxPool (3×3, stride 2)
- Conv2D (128 filter, 3×3, stride 1, ReLU) + BatchNorm
- Conv2D (256 filter, 3×3, stride 1, ReLU) + BatchNorm
- Conv2D (512 filter, 3×3, stride 1, ReLU) + BatchNorm + AvgPool (2×2, stride 2)
- Flatten
- Dense (512) + LeakyReLU + LayerNorm
- Dense (256) + LeakyReLU + LayerNorm
- Dense (17, softmax dengan dtype float32)

### Prerequisite untuk Profiling

**Install Requirement**:
1. **Python package**: `pip install tensorboard_plugin_profile`
2. **CUDA Profiling Toolkit (libcupti)**:
   - **Ubuntu**: `sudo apt-get install libcupti-dev`
   - **Windows**: Copy DLL dari CUDA extras folder ke bin/lib directory, enable GPU profiling untuk semua user via NVIDIA Control Panel

### Interface Profiling

**Enable**: Set `profile_batch=[10, 20]` dalam TensorBoard callback untuk profile batch 10-20.

**Metrik Overview Page**:
- **Input time**: Operasi pembacaan data (tf.data.Dataset)
- **Host compute time**: Komputasi CPU (model-related)
- **Device-to-device time**: Transfer data CPU → GPU
- **Kernel launch time**: CPU meluncurkan GPU kernel
- **Device compute time**: Komputasi GPU (ideal: proporsi tertinggi)
- **Device collective communication**: Multi-GPU/multi-node communication
- **Other time**: Kompilasi, output, miscellaneous

**Informasi Tambahan**:
- % TensorFlow ops ditempatkan di host vs device
- Data type breakdown (float16 vs float32)
- Utilisasi TensorCore (specialized matrix multiplication hardware)

**Recommendation Section**: Saran otomatis untuk perbaikan performa berdasarkan hasil profiling.

### Optimisasi 1: tf.data Pipeline

**Masalah yang Teridentifikasi**:
- Input time tinggi (data-bound)
- Pembacaan data sequential dari disk

**Solusi**:
1. **Prefetching**: `dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)` - Pipeline menyiapkan batch berikutnya saat model melatih current batch
2. **Parallel map**: `dataset.map(fn, num_parallel_calls=tf.data.AUTOTUNE)` - Decoding gambar di-parallelize
3. **Dedicated GPU thread**: Set environment variable `TF_GPU_THREAD_MODE=gpu_private` untuk allocate dedicated CPU thread untuk GPU kernel launch, preventing delay dari CPU contention

**Setting Environment Variable**:
- **Linux**: `export TF_GPU_THREAD_MODE=gpu_private` di terminal sebelum start notebook server
- **Windows**: System setting → Environment variable → Tambah `TF_GPU_THREAD_MODE=gpu_private`
- **Conda**: `conda env config vars set TF_GPU_THREAD_MODE=gpu_private`, deactivate/reactivate environment

**Kritis**: Restart notebook server setelah set environment variable (child process inherit environment).

### Optimisasi 2: Mixed Precision Training

**Konsep**: Gunakan 16-bit floating point (float16) untuk komputasi, 32-bit (float32) untuk penyimpanan variable. Benefit:
- **Speed**: Operasi float16 lebih cepat di GPU modern dengan Tensor Core
- **Memory**: Pengurangan memori 50% (5.48 GB → 1.25 GB dalam contoh)
- **Precision preservation**: Variable disimpan sebagai float32 untuk stable gradient update

**Enable di Keras**:
```python
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
```

**Data Type Distribution**:
- **Input**: float16
- **Output**: float16
- **Variable** (weight, bias): float32
- **Komputasi**: float16 dimana possible

**Loss Scaling**: Automatically handled oleh Keras. `LossScaleOptimizer` wrap optimizer, scale loss untuk avoid underflow (float16 punya dynamic range lebih kecil dari float32: max 65.504 vs 3.4×10³⁸). Gradient di-scale up saat forward pass, di-scale down saat weight update.

**bfloat16**: Alternatif format 16-bit dari Google, same exponent range seperti float32 tapi lower precision. Better untuk TPU dan GPU dengan Tensor Core.

### Hasil

**Perbaikan Time**: Input time drastis berkurang (data bukan lagi bottleneck) setelah optimisasi pipeline. Device compute time slightly improved dengan mixed precision.

**Perbaikan Memory**: Pengurangan 76% (5.48 GB → 1.25 GB) dengan mixed precision training.

### Memory Profile View

**Heap vs Stack**:
- **Stack**: Function call tracking (automatically managed, popped setelah function return)
- **Heap**: Object dan variable yang dibuat saat execution (persist sampai garbage collected)

**Analysis**: Linear heap growth = tidak ada memory leak. Spike = temporary allocation saat operasi spesifik.

**Operation Breakdown Table**: Shows memory usage per TensorFlow operation. Contoh: Dense layer [115200, 512] menggunakan paling banyak memori (first Dense setelah Flatten bottleneck).

### Trace Viewer

**Visualisasi longitudinal** menunjukkan kapan dan bagaimana operasi dijalankan di CPU/GPU sepanjang waktu. Fitur:
- **Timeline**: X-axis = waktu, Y-axis = execution stream berbeda (CPU core, GPU, GPU private thread)
- **Operation block**: Color-coded bar mengindikasikan operation type dan duration
- **Zoom/pan**: Inspect interval waktu spesifik
- **Detail**: Click operasi untuk lihat metadata (duration, input/output shape)

**Insight**: Confirm operasi tf.data berjalan di CPU, operasi model di GPU, dedicated thread menangani kernel launch.

---

## Bagian 5: Visualisasi Word Vector

### GloVe Vectors

**GloVe (Global Vectors)** dari Stanford: Pretrained word embedding yang dipelajari dari large corpus. Berbeda dengan Skip-gram/CBOW (local context only), GloVe menggabungkan global co-occurrence statistic.

**Variant**:
- Wikipedia 2014 + Gigaword 5: 6B token, 400K vocab, 50/100/200/300D
- Common Crawl (42B): 42B token, 1.9M vocab, 300D
- Common Crawl (840B): 840B token, 2.2M vocab, 300D (cased)
- Twitter: 2B tweet, 27B token, 1.2M vocab, 25/50/100/200D

Bab menggunakan versi Wikipedia 50D (smallest).

### Data Pipeline

1. **Load GloVe**: `pd.read_csv('glove.6B.50d.txt')` return DataFrame dengan word index, 50 embedding column
2. **Load IMDB review**: `tfds.load('imdb_reviews')` untuk corpus
3. **Extract vocabulary**: Counter untuk find 5.000 word paling common dalam review
4. **Find intersection**: `df.loc[df.index.isin(most_common_words)]` berikan ~3.600 token common

### Saving untuk TensorBoard

**Langkah**:
1. **Create variable**: `weights = tf.Variable(df_common.values)` (shape [vocab_size, embedding_dim])
2. **Save checkpoint**: `tf.train.Checkpoint(embedding=weights).save(path)` save weight
3. **Save metadata**: TSV file dengan satu token per baris (row i = token untuk embedding row i)
4. **Configure projector**:
   ```python
   config = projector.ProjectorConfig()
   embedding = config.embeddings.add()
   embedding.metadata_path = 'metadata.tsv'
   projector.visualize_embeddings(log_dir, config)
   ```

**ProjectorConfig**: Mengandung model_checkpoint_directory, embedding configuration.

**EmbeddingInfo**: Mengandung tensor_name, metadata_path untuk label.

### Dimensionality Reduction

**Problem**: Vektor 50D tidak bisa divisualisasikan langsung. Perlu proyeksi ke space 2D/3D.

**Algoritma**:
- **PCA (Principal Component Analysis)**: Proyeksi linear yang maksimalkan variance
- **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: Non-linear, preserve local structure
- **UMAP (Uniform Manifold Approximation and Projection)**: Cepat, preserve both local dan global structure

TensorBoard menyediakan ketiga dengan adjustable hyperparameter (perplexity untuk t-SNE, neighbor untuk UMAP, dll).

### Fitur Visualisasi

**Dashboard Control**:
- **Dimensionality**: Toggle 2D ↔ 3D visualization
- **Algorithm selection**: Pilih PCA/t-SNE/UMAP dengan parameter slider
- **Hover inspection**: Mouse over point reveal word label
- **Search**: Regex-based search highlight match

**Contoh Analysis**: Regex `(?:fred|larry|mrs\.|mr\.|michelle|comedy|theater)` highlight nama orang dan movie-related term, showing semantic clustering (nama orang cluster bersama, genre word cluster terpisah).

---

## Program-Program Implementasi

### Program 1: Logging Gambar ke TensorBoard

```python
from datetime import datetime
import tensorflow as tf

# Setup logging directory dengan timestamp
log_timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
image_logdir = f"./logs/data_{log_timestamp}/train"

# Create writer
image_writer = tf.summary.create_file_writer(image_logdir)

# Log gambar individual dengan class label
id2label_map = {
    0: "Kaos/atas", 1: "Celana", 2: "Pullover", 3: "Gaun",
    4: "Mantel", 5: "Sandal", 6: "Kemeja", 7: "Sneaker",
    8: "Tas", 9: "Boot pergelangan kaki"
}

with image_writer.as_default():
    # Log 10 gambar individual dengan grouping berbasis class
    for data in fashion_ds["train"].batch(1).take(10):
        label = int(data["label"].numpy())
        tf.summary.image(
            id2label_map[label],  # Tag dengan nama class
            data["image"],
            max_outputs=1,
            step=0
        )
    
    # Log batch 20 gambar bersama
    for data in fashion_ds["train"].batch(20).take(1):
        tf.summary.image(
            "Training batch",
            data["image"],
            max_outputs=20,
            step=0
        )

# Luncurkan TensorBoard
%load_ext tensorboard
%tensorboard --logdir ./logs --port 6006
```

**Penjelasan**: Writer dibuat dengan direktori log yang ditimestamp. Gambar individual di-log dengan class label sebagai tag (dikelompokkan berdasarkan class di dashboard). Batch 20 ditampilkan bersama. TensorBoard diluncurkan inline di Jupyter notebook.

---

### Program 2: Training Model dengan TensorBoard Callback

```python
import os
from datetime import datetime

# Define model
dense_model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
dense_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), strides=2, padding='same',
                           activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])
conv_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Setup direktori log terpisah
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
dense_log_dir = os.path.join("logs", f"dense_{timestamp}")
conv_log_dir = os.path.join("logs", f"conv_{timestamp}")

# Siapkan data
batch_size = 64
tr_ds_flat, v_ds_flat, _ = get_train_valid_test_datasets(
    fashion_ds, batch_size, flatten_images=True
)
tr_ds_img, v_ds_img, _ = get_train_valid_test_datasets(
    fashion_ds, batch_size, flatten_images=False
)

# Train dense model
dense_tb = tf.keras.callbacks.TensorBoard(
    log_dir=dense_log_dir,
    histogram_freq=0,  # Tidak ada histogram
    profile_batch=0    # Tidak ada profiling
)
dense_model.fit(tr_ds_flat, validation_data=v_ds_flat,
                epochs=10, callbacks=[dense_tb])

# Train CNN dengan activation histogram
conv_tb = tf.keras.callbacks.TensorBoard(
    log_dir=conv_log_dir,
    histogram_freq=2,  # Log histogram setiap 2 epoch
    profile_batch=0
)
conv_model.fit(tr_ds_img, validation_data=v_ds_img,
               epochs=10, callbacks=[conv_tb])

# View hasil (kedua run ditampilkan di same dashboard)
%tensorboard --logdir ./logs
```

**Penjelasan**: Dua model di-train dengan direktori log terpisah, enabling side-by-side comparison. Dense model track hanya metrik, CNN track metrik + activation histogram setiap 2 epoch. Run yang ditimestamp prevent directory conflict. TensorBoard automatically detect dan display kedua run.

---

### Program 3: Custom Metric Logging

```python
import numpy as np
import tensorflow as tf

# Define model (standard vs batch norm)
dense_standard = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(256, activation='relu', name='log_layer'),
    tf.keras.layers.Dense(10, activation='softmax')
])
dense_standard.compile(loss='sparse_categorical_crossentropy',
                       optimizer='adam', metrics=['accuracy'])

dense_bn = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu', name='log_layer_bn'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])
dense_bn.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam', metrics=['accuracy'])

# Custom training function
def train_model(model, dataset, log_dir, log_layer_name, epochs):
    writer = tf.summary.create_file_writer(log_dir)
    step = 0
    
    with writer.as_default():
        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
            for batch in dataset:
                # Training step
                model.train_on_batch(*batch)
                
                # Extract weight dari target layer
                weights = model.get_layer(log_layer_name).get_weights()[0]
                
                # Hitung dan log statistik
                tf.summary.scalar('mean_weights',
                                 np.mean(np.abs(weights)), step=step)
                tf.summary.scalar('std_weights',
                                 np.std(np.abs(weights)), step=step)
                
                # Flush ke disk
                writer.flush()
                step += 1

# Train kedua model
tr_ds, _, _ = get_train_valid_test_datasets(
    fashion_ds, batch_size=64, flatten_images=True
)

train_model(dense_standard, tr_ds, 'logs/standard', 'log_layer', 5)
train_model(dense_bn, tr_ds, 'logs/bn', 'log_layer_bn', 5)
```

**Penjelasan**: Custom training loop manually log weight statistic per batch. `train_on_batch()` perform single gradient update, weight extracted dari layer spesifik, statistik dihitung dan di-log via tf.summary.scalar(). Writer.flush() ensure immediate disk write. Hasil menunjukkan batch norm increase weight variability.

---

### Program 4: Profiling Performa dan Optimisasi

```python
import os
from tensorflow.keras import mixed_precision

# Optimized data pipeline
def get_flower_datasets_optimized(image_dir, batch_size):
    dataset = tf.data.Dataset.list_files(
        os.path.join(image_dir, '*.jpg'), shuffle=False
    )
    
    def get_image_and_label(file_path):
        tokens = tf.strings.split(file_path, os.path.sep)
        label = (tf.strings.to_number(
            tf.strings.split(tf.strings.split(tokens[-1], '.')[0], '_')[-1]
        ) - 1) // 80
        
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        return tf.image.resize(img, [64, 64]), label
    
    # Parallel map + prefetch optimization
    dataset = dataset.map(
        get_image_and_label,
        num_parallel_calls=tf.data.AUTOTUNE
    ).shuffle(400)
    
    valid_ds = dataset.take(250).batch(batch_size)
    train_ds = dataset.skip(250).batch(batch_size).prefetch(
        tf.data.AUTOTUNE
    )
    
    return train_ds, valid_ds

# Enable mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Rebuild model dengan mixed precision
conv_model = get_cnn_model()  # Sama arsitektur

# Verify data type
print(f"Input dtype: {conv_model.layers[0].input.dtype}")       # float16
print(f"Variable dtype: {conv_model.layers[0].trainable_variables[0].dtype}")  # float32
print(f"Output dtype: {conv_model.layers[0].output.dtype}")     # float16

# Profile optimized model
tr_ds, v_ds = get_flower_datasets_optimized('data/17flowers/jpg', batch_size=32)

tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/optimized',
    profile_batch=[10, 20]
)

conv_model.fit(tr_ds, validation_data=v_ds, epochs=2, callbacks=[tb_callback])

# Bandingkan profile: pengurangan memori 76% (5.48GB → 1.25GB)
```

**Penjelasan**: Optimisasi include parallel map (decode gambar simultaneously), prefetch (siapkan batch saat model train), dan mixed precision (float16 ops, float32 variable). Environment variable TF_GPU_THREAD_MODE set terpisah. Profiling menunjukkan pengurangan memori dramatis dan perbaikan input time. Trace viewer confirm GPU utilization optimization.

---

### Program 5: Visualisasi Word Vector

```python
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from collections import Counter
from tensorboard.plugins import projector

# Load GloVe vector
df = pd.read_csv(
    'data/glove.6B.50d.txt',
    header=None,
    index_col=0,
    sep=None,
    error_bad_lines=False,
    encoding='utf-8'
)

# Load IMDB review dan extract vocabulary
review_ds = tfds.load('imdb_reviews')
corpus = []
for data in review_ds["train"]:
    text = str(np.char.decode(data["text"].numpy(), encoding='utf-8')).lower()
    corpus.append(text)

corpus_text = " ".join(corpus)
counter = Counter(corpus_text.split())
most_common_words = [w for w, _ in counter.most_common(5000)]

# Temukan token common
df_common = df.loc[df.index.isin(most_common_words)]
print(f"Token common: {len(df_common)}")  # ~3.600

# Save embedding dan metadata
log_dir = 'logs/embeddings'
weights = tf.Variable(df_common.values)
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Save metadata (token)
with open(os.path.join(log_dir, 'metadata.tsv'), 'w') as f:
    for word in df_common.index:
        f.write(word + '\n')

# Configure projector
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

# Luncurkan TensorBoard (must gunakan exact directory)
%tensorboard --logdir logs/embeddings/ --port 6007

# Contoh regex search: highlight word spesifik
# (?:fred|larry|comedy|theater|loving|sadistic|marvelous)
```

**Penjelasan**: Vektor GloVe di-load sebagai DataFrame, IMDB vocabulary extracted via Counter, intersection dihitung. Embedding di-save sebagai TensorFlow checkpoint, metadata TSV contain token yang sesuai. Projector dikonfigurasi dan divisualisasikan. Dashboard provide PCA/t-SNE/UMAP dengan adjustable parameter, hover inspection, regex search highlighting. Relationship semantik visible (word similar cluster bersama).

---

## Kesimpulan

TensorBoard adalah tool indispensable untuk modern ML development, menyediakan comprehensive visibility ke training process. Kapabilitas key: real-time metrics tracking enable early detection training issue, activation histogram reveal layer convergence pattern, profiling identify bottleneck (memory, computation, data pipeline), custom logging support arbitrary metrik via tf.summary API, dan embedding visualization enable high-dimensional data analysis. Optimisasi yang dipelajari (mixed precision training, parallel data loading, dedicated GPU thread) demonstrate bagaimana profiling insight langsung translate ke actionable improvement. TensorBoard extensibility (HParams untuk hyperparameter optimization, Debugger v2 untuk numerical issue, What-If Tool untuk interpretability) menjadikan comprehensive platform untuk full ML lifecycle—dari data exploration hingga production monitoring.
Z